### HDF5 and Plotly

This notebook will give an overview of using the excellent [HDF5 Data Format](https://www.hdfgroup.org/HDF5/) for high performance computing and [Plotly](https://plot.ly/) to graph data stored in this files.

For those unfamilar with the HDF5 file format:

HDF5 is a data model, library, and file format for storing and managing data. It supports an unlimited variety of datatypes, and is designed for flexible and efficient I/O and for high volume and complex data. HDF5 is portable and is extensible, allowing applications to evolve in their use of HDF5. The HDF5 Technology suite includes tools and applications for managing, manipulating, viewing, and analyzing data in the HDF5 format.

-- [The HDF5 Group](https://www.hdfgroup.org/HDF5/)

The HDF group has some great reasons to use their files - namely that it works great with all kind of data. You can [read more here.](https://www.hdfgroup.org/why_hdf/)

In [34]:
import pandas as pd
from IPython.display import display
import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout, Data, Figure, Heatmap
import plotly.tools as tls
import numpy as np

The dataset that we'll be using is data from [NYC's open data portal](https://nycopendata.socrata.com/data). We'll be exploring a 100mb dataset covering traffic accidents in NYC. While we are capable of fitting this data into memory, the HDF5 file format has some unique affordances that allow us to query and save data in convenient ways.

Now the first thing we'll want to do is open up an access point to this HDF5 file, doing so is simple because pandas provides ready access to doing so.

In [5]:
pd.set_option('io.hdf.default_format','table')

In [6]:
store = pd.HDFStore('nypd_motors.h5')

Now that we've opened up our store, let's start storing some data

In [ ]:
df = pd.read_csv('NYPD_motor_collisions.csv', parse_dates=['DATE'])
df.columns = [col.lower().replace(" ", "_") for col in df.columns]
store.append("nypd", df,format='table',data_columns=True)

In [7]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: nypd_motors.h5
/nypd            frame_table  (typ->appendable,nrows->596990,ncols->29,indexers->[index],dc->[DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5])

In [ ]:
# store.close()

One thing that's nice about the HDF5 file is that it's kind of like a key value store. It's simple to use, and allows you to store things just like you might in a file system type hierarchy.

What's awesome about the HDF5 format is that it's almost like a miniature file system. It supports hierarchical data and is accessed like a python dictionary.

In [9]:
store.get_storer("df")

In [10]:
store.select("nypd").head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2015-06-02,13:48,MANHATTAN,10038,40.711780,-73.999701,"(40.7117796, -73.9997006)",ST JAMES PLACE,MADISON STREET,NaN,...,Unspecified,NaN,NaN,NaN,3232026,VAN,VAN,NaN,NaN,NaN
1,2015-06-02,13:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Turning Improperly,NaN,NaN,NaN,3232021,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
2,2015-06-02,13:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1200 WATERS PLACE - PARKING LOT,...,Unspecified,NaN,NaN,NaN,3232261,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3,2015-06-02,13:40,MANHATTAN,10004,40.706701,-74.016047,"(40.7067007, -74.0160467)",WEST STREET,MORRIS STREET,NaN,...,Unspecified,NaN,NaN,NaN,3232015,UNKNOWN,PASSENGER VEHICLE,NaN,NaN,NaN
4,2015-06-02,13:38,NaN,NaN,NaN,NaN,NaN,WOOLLEY AVENUE,GURDON STREET,NaN,...,Other Vehicular,NaN,NaN,NaN,3233372,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


In [11]:
boroughs = store.select("nypd", "columns=['BOROUGH']")

In [12]:
boroughs['COUNT'] = 1
borough_groups = boroughs.groupby('BOROUGH')

In [13]:
borough_groups.sum().index

Index([u'BRONX', u'BROOKLYN', u'MANHATTAN', u'QUEENS', u'STATEN ISLAND'], dtype='object')

In [14]:
py.iplot({
        'data':[Bar(y=borough_groups.sum()['COUNT'], x=borough_groups.sum().index)],
    })

/Users/bill_chambers/.virtualenvs/plotly-notebook/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [15]:
dates_borough = store.select("nypd", "columns=['DATE', 'BOROUGH']").sort('DATE')

In [16]:
dates_borough['COUNT'] = 1

In [17]:
date_borough_sum = dates_borough.groupby(['BOROUGH', "DATE"]).sum()
date_borough_sum.head()

COUNT
BOROUGH DATE             
BRONX   2012-07-01     39
        2012-07-02     71
        2012-07-03     73
        2012-07-04     51
        2012-07-05     60

In [18]:
data = []
for g, df in date_borough_sum.reset_index().groupby('BOROUGH'):
    data.append(Scatter(x= df.DATE, y=df.COUNT,name=g))

In [19]:
py.iplot(Data(data), filename='nypd_crashes/over_time')

/Users/bill_chambers/.virtualenvs/plotly-notebook/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



Luckily for us, while this graph is a bit of a mess, we can still zoom in on specific times and ranges. This makes plotly perfect for exploring datasets. You can create a high level visual of the data then zoom into a more detailed level.

See below where using the above graph I could zoom in on a particular point and anontate it for future investigation.

In [20]:
tls.embed("https://plot.ly/~bill_chambers/274")

In [21]:
car_types = store.select("nypd", "columns=['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']")
car_types['COUNT'] = 1

In [22]:
code_1 = car_types.groupby('VEHICLE TYPE CODE 1').sum()
code_2 = car_types.groupby('VEHICLE TYPE CODE 2').sum()

In [23]:
data = Data([
        Bar(x=code_1.index, y=code_1.COUNT,name='First Vehicle Type'),
        Bar(x=code_2.index, y=code_2.COUNT,name='Second Vehicle Type')
     ])

In [24]:
py.iplot(Figure(data=data, layout=Layout(barmode='group')))

/Users/bill_chambers/.virtualenvs/plotly-notebook/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



No big surprises here, we can see that passenger vehicles, likely being the most prevalent vehicles, are the ones involved in the most accidents for the first and second vehicles. However this does make for some more interesting questions, does this extrapolate to each vehicle class. That is, do all kinds of vehicles hit all other vehicles in more or less the same frequency? 

Let's explore large commercial vehicles.

In [25]:
large_vehicles = car_types.groupby(
    'VEHICLE TYPE CODE 1'
).get_group(
    'LARGE COM VEH(6 OR MORE TIRES)'
).groupby('VEHICLE TYPE CODE 2').sum()

In [26]:
py.iplot(Data([Bar(x=large_vehicles.index,y=large_vehicles.COUNT)]))

/Users/bill_chambers/.virtualenvs/plotly-notebook/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



At first glance it seems alright, but it's worth more exploration - let's Z-Score the data and compare their scores.

In [27]:
large_vehicles.head()

,COUNT
VEHICLE TYPE CODE 2,
AMBULANCE,9
BICYCLE,98
BUS,151
FIRE TRUCK,6
LARGE COM VEH(6 OR MORE TIRES),878


In [28]:
code_2.head()

,COUNT
VEHICLE TYPE CODE 2,
AMBULANCE,842
BICYCLE,13891
BUS,8935
FIRE TRUCK,412
LARGE COM VEH(6 OR MORE TIRES),10299


In [29]:
def z_score(df):
    df['zscore'] = ((df.COUNT - df.COUNT.mean())/df.COUNT.std())
    return df

In [30]:
data = Data([
        Bar(x=z_score(code_2).index,y=z_score(code_2).zscore, name='All Vehicles'),
        Bar(x=z_score(large_vehicles).index,y=z_score(large_vehicles).zscore,name='Large Vehicles'),
        
     ])

In [31]:
py.iplot(data, name='nypd_crashes/large vs all vehicles')

/Users/bill_chambers/.virtualenvs/plotly-notebook/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



We can see that things are relatively similar, except that large vehicles seem to hit large vehicles much more than most others. This could warrant further investigation.

While grouped bar charts can be useful for these kinds of comparisons, it can be great to visualize this data with heatmaps as well. We can create one of these by creation a contingency table or cross tabulation.

In [35]:
cont_table = pd.crosstab(car_types['VEHICLE TYPE CODE 1'], car_types['VEHICLE TYPE CODE 2']).apply(np.log)

Because of the different magnitudes of data, I decided to log scale it.

In [36]:
py.iplot(Data([
            Heatmap(z = cont_table.values, x=cont_table.columns, y=cont_table.index, colorscale='Jet')
        ]),filename='nypd_crashes/vehicle to vehicle heatmap')

/Users/bill_chambers/.virtualenvs/plotly-notebook/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



With this we are able to see more interesting nuances in the data. For instance taxis seems to have lots of accidents with other taxis, while vans and station wagons also seem to have many accidents.

There's clearly a lot to explore in this dataset.